### Prompt Engineering 01 MENTAL HEALTH ###

In [ ]:
import os
import pandas as pd
import numpy as np

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt, MentalHealth
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, create_messages
from llmt.performance import Performance

In [ ]:
# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df = pd.read_parquet(test_file)
# Filter the labeled data
df = df.loc[df['dset'] == 'train'].\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})


display(df.head())
print(df.shape)

### Define the prompts and the response format ###

In [ ]:
mental_health_business_definition = Prompt().load(prompt_name='mental_health_definition')
system_prompt = Prompt().load(prompt_name='mental_health_system_01')

system_prompt += f' according to the following definition: {mental_health_business_definition}'

# For the user prompt, we want to include the name of the business and the description
def create_user_prompt(name: str, description: str):
    user_prompt = f"""
        The business or provider {name} is described as: {description} 
        Does the business provide mental or behavioral health care services?
        """
    return process_prompt(user_prompt)

In [ ]:
# Create an example for messages to be sent
idx = 2
df_idx = df.iloc[idx]
display(df_idx)
user_prompt = create_user_prompt(name=df_idx['name'], description=df_idx['description'])
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
print(*messages, sep='\n\n')

### Send messages to the model ###

In [ ]:
model = 'gpt-4o'
temperature = 0
response_format = MentalHealth
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
client = OpenAI().create_client()
output = OpenAI().send_messages(messages=messages,
                                model=model,
                                temperature=temperature,
                                response_format=response_format,
                                client=client)
# key = 'pred_mh'
# output.update({key: 1 if output.get(key) == True else 0})

In [ ]:
print(output)

#### Process predictions for all samples ###

In [ ]:
model = 'gpt-4o'
temperature = 0
response_format = MentalHealth
client = OpenAI().create_client()

In [ ]:
company_id_list = sorted(list(df['id'].unique()))
results_df_list = []
for c, company_id in enumerate(company_id_list):
    if (c + 1) % 20 == 0:
        print(f'Sending description {c + 1} / {len(company_id_list)} to the model')
    df_id = df.loc[df['id'] == company_id]
    user_prompt_id = create_user_prompt(name=df_id['name'].values[0], 
                                        description=df_id['description'].values[0])
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt_id)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=response_format,
                                    client=client)
    # Replace the boolean with binary outcome prediction
    key = 'pred_mh'
    output.update({key: 1 if output.get(key) == True else 0})
    results_df_list.append(df_id.assign(**output))
results_df = pd.concat(results_df_list, axis=0, ignore_index=True)
# Save the results
results_file_name = 'mh_prompt_01_250418.parquet'
results_file = os.path.join(data_dir, results_file_name)
results_df.to_parquet(results_file)

In [ ]:
results_df.head()

In [ ]:
display(Performance(data=results_df).\
        binary_performance(true_col='mental_health', pred_col='pred_mh'))